In [1]:
# load packages
import os
import tempfile
import torch

import planetary_computer
from torch.utils.data import DataLoader

import rioxarray as rxr
import matplotlib.pyplot as plt
from rioxarray.merge import merge_arrays
import geopandas as gpd

from torchgeo.datasets import NAIP, ChesapeakeDE, stack_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

In [2]:
# download NAIP imagery
naip_root = os.path.join(tempfile.gettempdir(), 'naip')
naip_url = (
    'https://naipeuwest.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/'
)
tiles = [
    'm_3807511_ne_18_060_20181104.tif',
    'm_3807511_se_18_060_20181104.tif',
    'm_3807512_nw_18_060_20180815.tif',
    'm_3807512_sw_18_060_20180815.tif',
]
for tile in tiles:
    url = planetary_computer.sign(naip_url + tile)
    download_url(url, naip_root, filename=tile)

naip = NAIP(naip_root)

Using downloaded and verified file: /var/folders/61/b_g16g9n7h1864j7mrkvshbw0000gn/T/naip/m_3807511_ne_18_060_20181104.tif
Using downloaded and verified file: /var/folders/61/b_g16g9n7h1864j7mrkvshbw0000gn/T/naip/m_3807511_se_18_060_20181104.tif
Using downloaded and verified file: /var/folders/61/b_g16g9n7h1864j7mrkvshbw0000gn/T/naip/m_3807512_nw_18_060_20180815.tif
Using downloaded and verified file: /var/folders/61/b_g16g9n7h1864j7mrkvshbw0000gn/T/naip/m_3807512_sw_18_060_20180815.tif


In [3]:
# download corresponding Chesapeake labels
chesapeake_root = os.path.join(tempfile.gettempdir(), 'chesapeake')
os.makedirs(chesapeake_root, exist_ok=True)
chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

In [4]:
# create an intersection dataset that can sample from both datasets simultaneously
dataset = naip & chesapeake

In [5]:
# create random sample
sampler = RandomGeoSampler(dataset, size=1000, length=10)

In [6]:
# create dataloader
dataloader = DataLoader(dataset, sampler=sampler, collate_fn=stack_samples)

| ID | GPU | MEM |
------------------
